
## How does a merger between airlines affect prices?
### Data Preparation

 - using the airline-tickets-usa dataset
 


In [1]:
import os
import sys
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")


In [2]:
# Import the prewritten helper functions
#from py_helper_functions import *


## Data Preparation Steps and Descriptive Statistics

In [3]:
#data = pd.read_stata(data_in + "originfinal-panel.dta")
data = pd.read_stata("https://osf.io/zw2h9/download")

In [4]:
data = data.loc[lambda x: (x["year"] == 2011) | (x["year"] == 2016)].reset_index(
    drop=True
)


In [5]:
# * create total number of passengers from shares
# * so we can get aggreagate shares

data["ptotalAA"] = data["shareAA"] * data["passengers"]
data["ptotalUS"] = data["shareUS"] * data["passengers"]
data["ptotallargest"] = data["sharelargest"] * data["passengers"]

data_agg = (
    data.groupby(["origin", "finaldest", "return", "year"])
    .agg(
        airports=("airports", "first"),
        return_sym=("return_sym", "first"),
        stops=("stops", "first"),
        ptotalAA=("ptotalAA", sum),
        ptotalUS=("ptotalUS", sum),
        ptotallargest=("ptotallargest", sum),
        passengers=("passengers", sum),
        itinfare=("itinfare", sum),
    )
    .reset_index()
)


In [6]:
data_agg["after"] = np.where(data_agg["year"] == 2016, 1, 0)
data_agg["before"] = np.where(data_agg["year"] == 2011, 1, 0)
data_agg["avgprice"] = data_agg["itinfare"] / data_agg["passengers"]
data_agg["shareAA"] = data_agg["ptotalAA"] / data_agg["passengers"]
data_agg["shareUS"] = data_agg["ptotalUS"] / data_agg["passengers"]
data_agg["sharelargest"] = data_agg["ptotallargest"] / data_agg["passengers"]
data_agg["AA"] = np.where(data_agg["shareAA"] > 0, 1, 0)
data_agg["US"] = np.where(data_agg["shareUS"] > 0, 1, 0)
data_agg["US"] = np.where(data_agg["shareUS"] > 0, 1, 0)
data_agg["AA_and_US"] = np.where(
    (data_agg["shareAA"] > 0) & (data_agg["shareUS"]) > 0, 1, 0
)
data_agg["AA_or_US"] = np.where(
    (data_agg["shareAA"] > 0) | (data_agg["shareUS"]) > 0, 1, 0
)


In [7]:
# create numeric ID for market
data_agg["market"] = (
    data_agg["origin"]
    + "_"
    + data_agg["finaldest"]
    + "_"
    + data_agg["return"].apply(str)
)


In [8]:
# passengers before and after
data_agg["pass_bef_id"] = np.where(
    data_agg["before"] == 1, data_agg["passengers"], np.nan
)
data_agg["pass_aft_id"] = np.where(
    data_agg["after"] == 1, data_agg["passengers"], np.nan
)

data_agg["pass_bef"] = data_agg.groupby("market")["pass_bef_id"].transform(np.nanmean)
data_agg["pass_aft"] = data_agg.groupby("market")["pass_aft_id"].transform(np.nanmean)


In [9]:
# balanced vs unbalanced part of panel
data_agg["balanced"] = data_agg.groupby("market")["origin"].transform("count") == 2


In [10]:
data_agg.groupby("balanced").agg(
    sum_passengers=("passengers", sum), n=("origin", "count")
)


,sum_passengers,n
balanced,,
False,561060,51516
True,54136421,226094


### Define treated and untreated markets
- treated: both AA and US present in the before period
- untreated: neither AA nor US present in the before period OR only AA or only US in before period

This definition of treated and untreated markets left some markets neither treated nor untreated: those with only American or only US Airways present in 2011. For the main analysis we **kept these in the data as untreated**.

In [11]:
data_agg = data_agg.merge(
    data_agg.loc[lambda x: x["balanced"] == 1]
    .assign(
        treated=lambda x: (x["AA_and_US"] == 1) & (x["before"] == 1),
        untreated=lambda x: (x["AA_and_US"] == 0) & (x["before"] == 1),
        smallmkt=lambda x: (x["passengers"] < 5000) & (x["before"] == 1),
    )
    .groupby("market")
    .agg(
        treated=("treated", max),
        untreated=("untreated", max),
        smallmkt=("smallmkt", max),
    )
    .reset_index(),
    on="market",
    how="left",
)


In [12]:
data_agg["lnavgp"] = np.where(
    np.isinf(np.log(data_agg["avgprice"])), np.nan, np.log(data_agg["avgprice"])
)
data_agg["d_lnavgp"] = data_agg.groupby("market")["lnavgp"].transform("diff")


## Describe

In [13]:
# describe yearly data
data_agg[["year", "passengers"]].groupby("year").describe(
    percentiles=[0.25, 0.5, 0.75, 0.9]
).round(0)


passengers                                                    
          count   mean     std  min  25%  50%   75%    90%      max
year                                                               
2011   141712.0  170.0   980.0  1.0  2.0  7.0  37.0  205.0  39853.0
2016   135898.0  225.0  1270.0  1.0  2.0  8.0  45.0  245.0  46881.0

In [14]:
data_agg.query('(origin=="JFK") & (finaldest=="LAX")')[
    ["market", "origin", "finaldest", "return", "year", "passengers"]
]


,market,origin,finaldest,return,year,passengers
139047,JFK_LAX_0,JFK,LAX,0,2011,31312
139048,JFK_LAX_0,JFK,LAX,0,2016,46881
139049,JFK_LAX_1,JFK,LAX,1,2011,37861
139050,JFK_LAX_1,JFK,LAX,1,2016,41927


In [15]:
data_agg.query("year == 2011")[["smallmkt", "passengers"]].groupby(
    "smallmkt"
).describe().round(0)


passengers                                                          
              count    mean     std     min     25%     50%      75%      max
smallmkt                                                                     
False         887.0  9813.0  5748.0  5003.0  5987.0  7908.0  11278.0  39853.0
True       112160.0   136.0   456.0     1.0     3.0    12.0     55.0   4994.0

In [16]:
# describe balanced
data_agg.groupby(["year", "balanced"])["passengers"].describe(percentiles=[]).round(1)


count   mean     std  min   50%      max
year balanced                                             
2011 False      28665.0    3.6    23.8  1.0   1.0   2051.0
     True      113047.0  211.9  1092.7  1.0  12.0  39853.0
2016 False      22851.0   20.0   178.1  1.0   1.0   4798.0
     True      113047.0  267.0  1386.3  1.0  13.0  46881.0

In [30]:
# describe treatment
data_agg.groupby(["year", "treated", "untreated"]).describe(percentiles=[]).round(1).T


year                2011               2016         
treated            False    True      False    True 
untreated          True     False     True     False
return   count  101418.0  11629.0  101418.0  11629.0
         mean        0.5      0.5       0.5      0.5
         std         0.5      0.5       0.5      0.5
         min         0.0      0.0       0.0      0.0
         50%         1.0      0.0       1.0      0.0
...                  ...      ...       ...      ...
d_lnavgp mean        NaN      NaN       0.1      0.0
         std         NaN      NaN       0.7      0.3
         min         NaN      NaN      -6.4     -2.4
         50%         NaN      NaN       0.1      0.0
         max         NaN      NaN       6.5      2.3

[150 rows x 4 columns]

In [18]:
# describe outcome
data_agg.query("before == 1")["avgprice"].describe().round(2)


count    141712.00
mean        455.75
std         276.49
min           0.00
25%         285.50
50%         413.40
75%         561.03
max        6054.00
Name: avgprice, dtype: float64

In [19]:
data_agg.query("avgprice == 0")["passengers"].describe().round(2)


count    843.00
mean       1.08
std        0.31
min        1.00
25%        1.00
50%        1.00
75%        1.00
max        3.00
Name: passengers, dtype: float64

In [20]:

data_agg.to_pickle("airline-workfile.pkl")


In [26]:
descriptive_stats_all = data_agg.describe(percentiles=[0.25, 0.5, 0.75]).round(2)


In [27]:
descriptive_stats_all.T

,count,mean,std,min,25%,50%,75%,max
return,277610.0,0.52,0.50,0.00,0.00,1.00,1.00,1.00
year,277610.0,2013.45,2.50,2011.00,2011.00,2011.00,2016.00,2016.00
airports,277610.0,4.51,1.43,2.00,3.00,5.00,5.00,9.00
return_sym,145027.0,0.78,0.41,0.00,1.00,1.00,1.00,1.00
stops,277610.0,1.30,0.52,0.00,1.00,1.00,2.00,3.00
ptotalAA,277610.0,26.54,305.59,0.00,0.00,0.00,1.00,24782.00
ptotalUS,277610.0,6.26,108.71,0.00,0.00,0.00,0.00,19264.00
ptotallargest,277610.0,125.10,693.56,1.00,2.00,6.00,28.00,28735.00
passengers,277610.0,197.03,1131.39,1.00,2.00,7.00,41.00,46881.00
itinfare,277610.0,36930.82,155802.90,0.00,978.00,3355.00,15796.75,10189694.00


In [28]:
latex_output_all = descriptive_stats_all.T.to_latex()


In [29]:
latex_output_all

'\\begin{tabular}{lrrrrrrrr}\n\\toprule\n & count & mean & std & min & 25% & 50% & 75% & max \\\\\n\\midrule\nreturn & 277610.000000 & 0.520000 & 0.500000 & 0.000000 & 0.000000 & 1.000000 & 1.000000 & 1.000000 \\\\\nyear & 277610.000000 & 2013.450000 & 2.500000 & 2011.000000 & 2011.000000 & 2011.000000 & 2016.000000 & 2016.000000 \\\\\nairports & 277610.000000 & 4.510000 & 1.430000 & 2.000000 & 3.000000 & 5.000000 & 5.000000 & 9.000000 \\\\\nreturn_sym & 145027.000000 & 0.780000 & 0.410000 & 0.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 \\\\\nstops & 277610.000000 & 1.300000 & 0.520000 & 0.000000 & 1.000000 & 1.000000 & 2.000000 & 3.000000 \\\\\nptotalAA & 277610.000000 & 26.540000 & 305.590000 & 0.000000 & 0.000000 & 0.000000 & 1.000000 & 24782.000000 \\\\\nptotalUS & 277610.000000 & 6.260000 & 108.710000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 19264.000000 \\\\\nptotallargest & 277610.000000 & 125.100000 & 693.560000 & 1.000000 & 2.000000 & 6.000000 & 28.000000 & 28735.0